# 02: Add Recipient Continents and Basic Cleaning

Countries and Continent List taken from - https://www.kaggle.com/datasets/hserdaraltan/countries-by-continent

Goals of 02_add_continents_and_clean

- Read CRS_energy_raw_merged.csv from "01_txt_to_csv_and_merge.ipynb"
- Remove entries with no CRS ID
- Remove entries with no financial records
- Add DonorContinent and RecipientContinent
- save file as "energy_full_text"
- check uniqueness- based on specific columns
- remove duplicated and save file useful for STM as "energy_unique_text.csv"

In [2]:
import pandas as pd
import os

## Load the Data

In [76]:
# Read the merged file
input_path = '/content/CRS_energy_raw_merged.csv'
df = pd.read_csv(input_path)
print(f"Loaded data: {len(df)} records")

Loaded data: 185358 records


### Remove entries with no CRS ID

In [77]:
# remove entries with no CRS ID
df_filtered = df.dropna(subset=['CrsID'])
print(f"Records after dropping missing CrsID: {len(df_filtered)}")

Records after dropping missing CrsID: 184926


### Remove entries with non financial records

In [78]:
# remove entries with no financial transactions

df_filtered = df_filtered[
    (df_filtered['USD_Commitment'] > 0) | (df_filtered['USD_Disbursement'] > 0) | (df_filtered['USD_Received'] > 0)
]

print(f"Records remaining after financial flow check: {len(df_filtered)}")

Records remaining after financial flow check: 169076


### Continent Mapping

In [79]:
# Read the countries-by-continent mapping file
continent_mapping_path = '/content/Countries_by_continents.csv'
continent_df = pd.read_csv(continent_mapping_path)

In [80]:
# Clean the mapping dataset if needed
continent_df = continent_df[['Country', 'Continent']].dropna()

In [81]:
# Create a dictionary for faster mapping
country_to_continent = dict(zip(continent_df['Country'], continent_df['Continent']))

In [82]:
# Map RecipientName to Continent
df_filtered['DonorContinent'] = df_filtered['DonorName'].map(country_to_continent)

In [83]:
# Map RecipientName to Continent
df_filtered['RecipientContinent'] = df_filtered['RecipientName'].map(country_to_continent)

In [84]:
# Manual corrections for country names not matched automatically
manual_updates = {
    'Korea': 'Asia',
    'Slovak Republic': 'Europe',
    'Chinese Taipei': 'Asia'
}

# Apply manual updates
for country, continent in manual_updates.items():
    df_filtered.loc[df_filtered['DonorName'] == country, 'DonorContinent'] = continent
    df_filtered.loc[df_filtered['RecipientName'] == country, 'RecipientContinent'] = continent

In [85]:
#manually fix Turkey

# First, fix encoding issue by renaming "TÃ¼rkiye" to "Turkey"
df_filtered['DonorName'] = df_filtered['DonorName'].replace('Türkiye', 'Turkey')
df_filtered['RecipientName'] = df_filtered['RecipientName'].replace('Türkiye', 'Turkey')

# Then manually update continent and region for Turkey
df_filtered.loc[df['DonorName'] == 'Turkey', 'DonorContinent'] = 'Asia'
df_filtered.loc[df['RecipientName'] == 'Turkey', 'RecipientContinent'] = 'Asia'

Save as csv

In [87]:
df_full = df_filtered.copy()
print(f"Complete data: {len(df_full)} total records")

Complete data: 169076 total records


In [89]:
df_full.to_csv('/content/energy_full_text.csv', index=False)

### Check uniqueness

In [90]:
# Create UID column by combining year and CRDID as strings
df_filtered['UID_long'] = df_filtered['Year'].astype(str) + "_" + df_filtered['CrsID'].astype(str) + "_" + df_filtered['LongDescription'].astype(str) + "_" + df_filtered['RegionName'].astype(str) + "_" + df_filtered['ClimateMitigation'].astype(str) + "_" + df_filtered['ClimateAdaptation'].astype(str)

In [91]:
df_filtered['UID_long'].nunique()

79324

There are 79324 projects which are unique.


Since later stages of topic modeling involve estimating the effects of Year, Region, Climate Mitigation, and Climate Adaptation, I have kept the uniqueness criteria (UID) to include:

- Year
- CrsID
- LongDescription
- RegionName
- ClimateMitigation
- ClimateAdaptation

This ensures that projects with the same description but differing in region or climate focus are preserved as distinct observations during text analysis. It prevents loss of important variation when estimating topic-covariate relationships later in the analysis.

In [95]:
df_text = df_filtered.drop_duplicates(subset=['UID_long'], keep='first')
print(f"Text data: {len(df_text)} unique records")

Text data: 79324 unique records


In [96]:
#drop UID_long column
df_text = df_text.drop(columns="UID_long")

In [98]:
df_text.to_csv('/content/energy_unique_text.csv', index=False)

Later during STM effect estimation (estimateEffect step), I have modelled topics based on Region, Climate Mitigation, and Adaptation labels.

If projects were merged/ dropped based only on Year, CRSID, LongDescription,
a lot of important variation across these fields would have been lost.

Keeping these fields in UID ensures that if a project has the same description (along with Year and ID) but is targeted to different regions or focuses on different climate goals, they are treated as distinct records.